# Building a Spam Filter
The goal of this project is to build a spam filter for SMS messages. Specifically, we will leverage the multinomial Naive Bayes algorithm to create a spam classifier.

Our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from this [link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on this page, where you can also find some of the authors' papers.

### Importing and exploring the dataset

In [1]:
import pandas as pd

In [2]:
sms_spam_collection = pd.read_csv('SMSSpamCollection', sep='\t', header=None,
                                  names=['Label','SMS'])

In [3]:
sms_spam_collection.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
sms_spam_collection.shape

(5572, 2)

In [5]:
sms_spam_collection['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

The dataset contains only two columns, with 5572 rows representing different messages. Spam messages are labeled as "spam" while non-spam messages are labeled as "ham." In the dataset, approximately 86.6% of the messages are non-spam, while 13.4% are spam.

### Creating a training and test set
Before creating a spam filter, it's very helpful to first think of a way of testing how well it works. When creating software, a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying new messages. To test the spam filter, we're first going to split our dataset into two categories:

- A **training set**, which we'll use to "train" the computer how to classify messages.
- A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

In [6]:
random_df = sms_spam_collection.sample(frac=1, random_state=1)

In [7]:
split = round(0.8 * len(random_df))

In [8]:
training = random_df.iloc[:split].reset_index()
test = random_df.iloc[split:].reset_index()

In [9]:
training['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [10]:
test['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The percentages of the training and test set are approximately similar to the same dataset.

### Letter Case and Punctuation
Prior to calculating probabilities, the data must first be cleaned to ensure words are consistent for ease of analysis. This would involve removing punctuation as well as making all words lowercase.

In [11]:
training['SMS'] = training['SMS'].str.replace('\W',' ').str.lower()

In [12]:
training['SMS'].head()

0                         yep  by the pretty sculpture
1        yes  princess  are you going to make me moan 
2                           welp apparently he retired
3                                              havent 
4    i forgot 2 ask ü all smth   there s a card on ...
Name: SMS, dtype: object

### Creating the Vocabulary
The next step is creating a table with all the unique words as columns and count the number of times each word appears in each sentence.

In [13]:
training['SMS'] = training['SMS'].str.split()
vocabulary = []
for sms in training['SMS']:
    for word in sms:
        vocabulary.append(word)

In [14]:
training['SMS'].head()

0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

In [15]:
vocabulary = set(vocabulary) #removes duplicates

In [16]:
vocabulary = list(vocabulary) # transform back to list

In [17]:
len(vocabulary)

7783

### The Final Training Set
After obtaining the vocabulary, the final step is in creating a column for each of the word in the training set. Prior to that though, we need to build a dictionary that contains the words as keys and then a list with the length of the total rows in the training dataset with the word counts as the values.

In [18]:
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [19]:
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)

In [20]:
training = pd.concat([training, word_counts_per_sms], axis=1)

In [21]:
training.head()

,index,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,1078,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4028,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,958,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4642,ham,[havent],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4674,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


### Calculating Constants First
For the Naive Bayes algorithm, it needs to know the probability values of the two equations below to classify new messages:

$P(Spam\,|\,w_1, w_2,..., w_n) \propto P(Spam) \cdot {\displaystyle \prod_{i=1}^{n} P(w_i\,|\,Spam)}$
$P(Ham\,|\,w_1, w_2,..., w_n) \propto P(Ham) \cdot {\displaystyle \prod_{i=1}^{n} P(w_i\,|\,Ham)}$

To calculate $P(w_i\,|\,Spam) \,\text{and}\, P(w_i\,|\,Ham)$, 
we need to use these equations:

$P(w_i\,|\,Spam) = \frac{N_{w_i|Spam} +\,\alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$

$P(w_i\,|\,Ham) = \frac{N_{w_i|Ham} +\,\alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}$

There are five constants in each equation: $P(Spam), P(Ham), N_{Spam}, N_{Ham}, N_{Vocabulary}$

We will use Laplace smoothing and set $\alpha = 1$.

In [22]:
p_spam = len(training[training['Label']=='spam']) / len(training)
p_ham = 1 - p_spam

In [23]:
n_spam = sum([len(i) for i in training[training['Label']=='spam']['SMS']])
n_ham = sum([len(i) for i in training[training['Label']!='spam']['SMS']])
n_vocab = len(vocabulary)
alpha = 1

### Calculating Parameters
$P(w_i\,|\,Spam)$ and $P(w_i\,|\,Ham)$ will vary depending on the individual words. However, the probability for each individual word is constant for every new message. Since we have 7,783 words in our vocabulary we would need to calculate a total of 15,566 probabilities, with each word being represented as $w_i$.

We need to initialize two dictionaries, one for the spam and one for ham. Each words will be the key, with the values being the probabilities.

In [24]:
spam_probs = {k:0 for k in vocabulary}
ham_probs = {k:0 for k in vocabulary}

Separating the ham and spam messages in the training datasets.

In [25]:
spam_training = training[training['Label']=='spam'].copy()
ham_training = training[training['Label']=='ham'].copy()

$P(w_i\,|\,Spam)$ and $P(w_i\,|\,Ham)$ are calculated for each word in the vocabulary. After calculating the probabilities, the probability values in the initial two dictionaries are updated.

In [31]:
for w in vocabulary:
    spam_count = spam_training[w].sum()
    ham_count = ham_training[w].sum()
    w_given_spam = (spam_count + alpha) / (n_spam + alpha * n_vocab)
    w_given_ham = (ham_count + alpha) / (n_ham + alpha * n_vocab)
    spam_probs[w] = w_given_spam
    ham_probs[w] = w_given_ham

### Classifying A New Message
Now that all the constants and parameters are calculated, the next step is in creating the spam filter. Note that not all the words new messages may not be part of the vocabulary. In this instance, we simply ignore these words when calculating the probabilities.

In [36]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_probs.keys():
            p_spam_given_message *= spam_probs[word]
        if word in ham_probs.keys():
            p_ham_given_message *= ham_probs[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [37]:
test_1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
test_2 = 'Sounds good, Tom, then see u there'

In [38]:
classify(test_1)

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [39]:
classify(test_2)

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


It looks like the spam filter is working as expected.

### Measuring the Spam Filter's Accuracy
We will notw try to determine how well the spam filter does on our test set of 1,114 messages. Note that the algorithm did not see the 1,114 messages, so every message from the test set is new from the perspective of the algorithm and thus reducing bias.

Firstly, the classify function will be modified to return the labels instead of printing them.

In [43]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam_probs:
            p_spam_given_message *= spam_probs[word]

        if word in ham_probs:
            p_ham_given_message *= ham_probs[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Using the modified function above, we can use it to create a new column in the test set. We can then measure the accuracy of our algorithm by comparing the `Label` column from the new `predicted` column.

In [44]:
test['predicted'] = test['SMS'].map(classify_test_set)

In [47]:
test.head()

,index,Label,SMS,predicted
0,2131,ham,Later i guess. I needa do mcat study too.,ham
1,3418,ham,But i haf enuff space got like 4 mb...,ham
2,3424,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,5393,ham,"All done, all handed in. Don't know if mega sh...",ham


In [50]:
correct = len(test[test['Label'] == test['predicted']])
total = len(test)
accuracy = correct/total

In [52]:
print('Total accuracy: {:.2%}'.format(accuracy))

Total accuracy: 98.74%


### Conclusion
The accuracy of the algorithm is better than expected. The initial goal was to get an accuracy over 80%. Future directions for this project include:
- Isolate the 14 messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
- Make the filtering process more complex by making the algorithm sensitive to letter case.